In [ ]:
import usgoc.evaluation.evaluate as ee

m, calibration_configs, h1, h2, h3, dims, train_ds, val_ds, test_ds = ee.evaluate(
  "GIN",
  convert_mode="atomic_blocks", limit_id="v127_d127_f127_p127",
  fold=0, repeat=0, dry=True, 
  return_models=True, return_calibration_configs=True, return_size_histograms=True,
  return_ds=True)

In [ ]:
import numpy as np
from usgoc.postprocessing.calibration import platt_scaling

preds1 = []
preds2 = []
trues1 = []
trues2 = []

for batch in test_ds:
  pred1, pred2 = m(batch[0])
  true1, true2 = batch[1]
  preds1.append(pred1)
  preds2.append(pred2)
  trues1.append(true1)
  trues2.append(true2)

preds1 = np.concatenate(preds1)
preds2 = np.concatenate(preds2)
trues1 = np.concatenate(trues1)
trues2 = np.concatenate(trues2)

temp1 = platt_scaling(preds1, trues1)
temp2 = platt_scaling(preds2, trues2)
preds1 = preds1 / temp1
preds2 = preds2 / temp2
temp1, temp2


In [ ]:
import tensorflow as tf
import numpy as np
import funcy as fy

def compute_pred_cumsum(pred):
  pred = tf.nn.softmax(pred).numpy()
  # print(np.round(pred[ri:rj] * 1000))
  pred_pi = np.argsort(pred)[:, ::-1]
  pred_sorted = np.take_along_axis(pred, pred_pi, axis=1)
  pred_cumsum = np.cumsum(pred_sorted, -1)
  return pred_sorted, pred_cumsum, pred_pi

ri, rj = 5, 10

def adaptive_calibration_scores(pred, true):
  pred_sorted, pred_cumsum, pred_pi = compute_pred_cumsum(pred)
  pred_pi_inv = np.argsort(pred_pi)
  # print(pred_pi_inv[ri:rj])
  # print(np.round(pred_cumsum[ri:rj] * 1000))
  true_cumsumidx = np.take_along_axis(
    pred_pi_inv, np.expand_dims(true, 1), axis=1)
  U = np.random.random_sample((pred_sorted.shape[0], 1))
  pred_cumsum = np.pad(pred_cumsum, ((0, 0), (1, 0)), "constant")
  # print(true[ri:rj])
  scores = np.take_along_axis(pred_cumsum, true_cumsumidx, axis=1)
  scores += np.take_along_axis(pred_sorted, true_cumsumidx, axis=1) * U
  scores = np.squeeze(scores)
  # print(np.round(scores[ri:rj] * 1000))
  return scores

scores1 = adaptive_calibration_scores(preds1, trues1)
scores2 = adaptive_calibration_scores(preds2, trues2)

n = scores1.shape[0]

def conformal_sets(pred, qhat):
  _, pred_cumsum, pred_pi = compute_pred_cumsum(pred)
  sizes = np.argmax(pred_cumsum > qhat, axis=1)
  pred_sets = [pred_pi[i][:(sizes[i] + 1)] for i in range(sizes.shape[0])]
  # for i in range(sizes.shape[0]):
  #   if len(pred_sets[i]) == 11:
  #     print("zero info:", pred[i], np.round(tf.nn.softmax(pred[i]).numpy() * 1000) / 10, qhat)
  return pred_sets

test_pred1, test_pred2 = m.predict(test_ds)
test_pred1 = test_pred1 / temp1
test_pred2 = test_pred2 / temp2
test_true1, test_true2 = list(test_ds)[0][1]
test_eval = m.evaluate(test_ds, return_dict=True)

def get_test_sets(alpha):
  q = np.ceil((n + 1) * (1 - alpha)) / n
  qhat1 = np.quantile(scores1, q, method="higher")
  qhat2 = np.quantile(scores2, q, method="higher")
  print()
  print("q", q, ", qhat1", qhat1, ", qhat2", qhat2)
  
  test_set1 = conformal_sets(test_pred1, qhat1)
  test_set2 = conformal_sets(test_pred2, qhat2)
  ms1 = fy.ljuxt(np.min, np.mean, np.std, np.median, np.max)(fy.lmap(len, test_set1))
  ms2 = fy.ljuxt(np.min, np.mean, np.std, np.median, np.max)(fy.lmap(len, test_set2))

  # acc1 = np.mean((tf.argmax(test_pred1, -1).numpy() == test_true1).astype(np.int32))
  setacc1 = np.mean([test_true1[i] in test_set1[i] for i in range(len(test_set1))])
  setacc2 = np.mean([test_true2[i] in test_set2[i] for i in range(len(test_set2))])
  combacc = np.mean([
    test_true1[i] in test_set1[i] and test_true2[i] in test_set2[i]
    for i in range(len(test_set1))
  ])

  print(ms1, test_eval["label1_accuracy"], setacc1)
  print(ms2, test_eval["label2_accuracy"], setacc2)
  print(test_eval["accuracy"], combacc)
  # print([(s, t) for s, t in zip(test_set2, test_true2.numpy()) if len(s) > 1])
  return test_set1, test_set2, setacc1, setacc2, ms1, ms2, combacc


alphas = [0.03, 0.05, 0.07, 0.1, 0.2, 0.5]
# alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
res = [get_test_sets(a) for a in alphas]


In [ ]:
import matplotlib.pyplot as plt

def plot_set_hist(a, test_set1, test_set2, setacc1, setacc2, ms1, ms2, combacc):
  combacc_diff = combacc - test_eval["accuracy"]
  print(f"alpha = {a}, combacc = {combacc*100:.2f}% (+{combacc_diff*100:.2f}%)")
  setacc1 *= 100
  setacc2 *= 100
  true_acc1 = test_eval["label1_accuracy"] * 100
  true_acc2 = test_eval["label2_accuracy"] * 100
  size1a = ms1[1]
  size2a = ms2[1]
  size1m = ms1[3]
  size2m = ms2[3]
  setacc1_diff = setacc1 - true_acc1
  setacc2_diff = setacc2 - true_acc2
  bins = 1 + np.arange(11)
  fig, (ax1, ax2) = plt.subplots(1, 2)
  fig.set_size_inches(6, 2)
  sizes1 = fy.lmap(len, test_set1)
  sizes2 = fy.lmap(len, test_set2)
  hm1 = np.max(np.unique(sizes1, return_counts=True)[1])
  hm2 = np.max(np.unique(sizes2, return_counts=True)[1])
  
  ax1.set_title(f"L1: {size1a:.2f}, {setacc1:.2f} (+{setacc1_diff:.2f})")
  ax1.set_xticks(bins)
  ax1.hist(sizes1, bins=bins)
  ax1.vlines(size1a, 0, hm1, colors="red")
  ax1.vlines(size1m, 0, hm1, colors="orange")
  
  ax2.set_xticks(bins)
  ax2.set_title(f"L2: {size2a:.2f}, {setacc2:.2f} (+{setacc2_diff:.2f})")
  ax2.hist(sizes2, bins=bins)
  ax2.vlines(size2a, 0, hm2, colors="red")
  ax2.vlines(size2m, 0, hm2, colors="orange")
  
  plt.show()

for a, r in zip(alphas, res): 
  plot_set_hist(a, *r)
